# Living Earth for Land Degradation
L4 class Editor: UI + Black Formatting + File Generation
---

This notebook provides an interactive way to inspect, edit, and regenerate selected Level‑4 classification classes in the 'l4_layers_LE4LD_modified.py' source file used in the implementation of LE4LD in Living Earth pipeline.

Authors: Audrey Lambiel, Mona Bonnier, Carole Planque, Gregory Giuliani

Contact: audrey.lambiel@unige.ch

---- 
**What it does**

Parses the source file into an AST to safely read class blocks, docstrings, and __init__ bodies.
Presents a UI (ipywidgets) where user select a class, edit per‑level output codes/descriptions and optional class boundaries, and adjust the base type (categorical vs continuous).
Generates a new class block using programmatic edits.
Uses Black formatting to ensure consistent, PEP8‑style code formatting:

Preview formatting via a temporary scaffold so Black can format even if the fragment would otherwise fail to parse.
Final Apply formatting of the entire file for uniform results.

**How it works**

The notebook reads the source file (SOURCE) into memory and builds a classes dict with metadata (name, base class, docstring, __init__ body, spans for replacement).
UI widgets let you enter/modify output codes and class boundaries at specific levels.
When you click Preview, the notebook constructs the updated class fragment and formats it with Black using a scaffold; when you click Apply, it splices the formatted block into the working text and then formats the entire file with Black before writing to OUTPUT.
Class metadata is rebuilt after Apply/Reset so previews always reflect current contents.

**Requirements**

Python: 3.9+ (Black’s library API and ast.unparse are available from 3.9).
Environment: Use a dedicated virtual environment (e.g., venv, Conda, ...) to keep dependencies isolated.

Please ensure the following Python packages are installed in your environment:
- `black`
- `ipywidgets`
- `ipykernel`

+ standard library (ast, re, textwrape, shutil, ...)

> If you use Conda, a typical environment might be created with:
```bash
conda create -n le4ld python=3.13.9 black ipywidgets ipykernel
conda activate le4ld
```

**Safety and backups**

The notebook creates a backup (SOURCE.bak) the first time you click on Apply and Write button. You can revert by replacing SOURCE with SOURCE.bak if needed. 

---

## Set up

In [ ]:
# modules
import re
import ast
import shutil
from collections import OrderedDict
import textwrap
import black
import ipywidgets as widgets
from IPython.display import display

# config
SOURCE = "C:/monalisa/l4_layers_LE4LD_test.py" # update as needed
OUTPUT = "C:/monalisa/l4_layers_LE4LD_custom.py" # update as needed

LEVEL_KEYS = list(range(100, 200, 5)) + [200] # define possible land degradation categories, per default 100-200 in steps of 5

## Process

In [ ]:
# Black helper + pretty‑printers
def format_with_black(source: str, line_length: int = 88) -> str:
    """
    Return the Black-formatted version of `source`.
    """
    mode = black.Mode(line_length=line_length)
    return black.format_str(source, mode=mode)

def _ordered_dict(d):
    """
    Return OrderedDict sorted by key ascending (stable output).
    """
    try:
        return OrderedDict(sorted(d.items(), key=lambda kv: kv[0]))
    except Exception:
        return OrderedDict(d.items())

def quote_str(s):
    """
    Return a Python-literal quoted string (safe repr).
    """
    return repr(str(s))

def format_output_codes_literal(d, indent="    "):
    """
    Format { code: ("label", "desc"), ... } as a multi-line Python dict literal.
    """
    if not d:
        return "{ }"
    od = _ordered_dict(d)
    lines = ["{"]
    for k, (label, desc) in od.items():
        lines.append(f'{indent}{k} : ({quote_str(label)}, {quote_str(desc)}),')
    lines[-1] = lines[-1].rstrip(",")
    lines.append("}")
    return "\n".join(lines)

def format_boundaries_literal(d, indent="    "):
    """
    Format { code: (min, max), ... } with None or numbers, multi-line.
    """
    if not d:
        return "{ }"
    od = _ordered_dict(d)

    def val_repr(x):
        return "None" if x is None or x == "" else str(x)

    lines = ["{"]
    for k, (mn, mx) in od.items():
        lines.append(f"{indent}{k} : ({val_repr(mn)}, {val_repr(mx)}),")
    lines[-1] = lines[-1].rstrip(",")
    lines.append("}")
    return "\n".join(lines)

In [ ]:
# Read source file
with open(SOURCE, "r", encoding="utf-8") as f:
    source_text = f.read()

# Build fast line->offset mapping for absolute slicing
_lines = source_text.splitlines(keepends=True)
_line_starts = []
offset = 0
for ln in _lines:
    _line_starts.append(offset)
    offset += len(ln)

# AST facilities
def span_to_slice(lineno, col, end_lineno, end_col):
    """
    Convert a (lineno, col) .. (end_lineno, end_col) span to absolute (start, end) offsets.
    """
    start = _line_starts[lineno - 1] + col
    end = _line_starts[end_lineno - 1] + end_col
    return start, end

def get_source_segment_by_span(node):
    """
    Return exact source text for an AST 'node' if span attributes are present.
    """
    if not hasattr(node, "lineno") or not hasattr(node, "end_lineno"):
        return ""
    s, e = span_to_slice(node.lineno, node.col_offset, node.end_lineno, node.end_col_offset)
    return source_text[s:e]

module = ast.parse(source_text)

def unparse_expr(expr):
    """
    Best-effort conversion from AST expression to a string.
    """
    try:
        return ast.unparse(expr)  # Python 3.9+
    except Exception:
        if isinstance(expr, ast.Name):
            return expr.id
        elif isinstance(expr, ast.Attribute):
            return f"{unparse_expr(expr.value)}.{expr.attr}"
        else:
            return "<expr>"

def get_base_class_string(cls: ast.ClassDef):
    bases = [unparse_expr(b) for b in cls.bases]
    return ", ".join(bases) if bases else ""

def get_class_docstring_block(cls: ast.ClassDef):
    """Return the exact docstring block (including quotes) if present."""
    if not cls.body:
        return ""
    first = cls.body[0]
    if isinstance(first, ast.Expr) and isinstance(first.value, ast.Constant) and isinstance(first.value.value, str):
        return get_source_segment_by_span(first)
    return ""

def find_init_func(cls: ast.ClassDef):
    for n in cls.body:
        if isinstance(n, ast.FunctionDef) and n.name == "__init__":
            return n
    return None

def get_init_body_text(init_func: ast.FunctionDef):
    """Return the body text of __init__ (without the header line)."""
    full = get_source_segment_by_span(init_func)
    newline_idx = full.find("\n")
    if newline_idx == -1:
        return ""
    return full[newline_idx + 1 :]

In [ ]:
# Build classes metadata
def build_classes_from_text(text: str) -> dict:
    """
    Parse 'text' to AST and rebuild the 'classes' metadata dict.
    """
    module_local = ast.parse(text)
    classes_local = {}

    lines_local = text.splitlines(keepends=True)
    line_starts_local = []
    off = 0
    for ln in lines_local:
        line_starts_local.append(off)
        off += len(ln)

    def span_to_slice_local(lineno, col, end_lineno, end_col):
        start = line_starts_local[lineno - 1] + col
        end = line_starts_local[end_lineno - 1] + end_col
        return start, end

    def get_source_segment_by_span_local(node):
        if not hasattr(node, "lineno") or not hasattr(node, "end_lineno"):
            return ""
        s, e = span_to_slice_local(node.lineno, node.col_offset, node.end_lineno, node.end_col_offset)
        return text[s:e]

    def get_class_docstring_block_local(cls: ast.ClassDef):
        if not cls.body:
            return ""
        first = cls.body[0]
        if isinstance(first, ast.Expr) and isinstance(first.value, ast.Constant) and isinstance(first.value.value, str):
            return get_source_segment_by_span_local(first)
        return ""

    def find_init_func_local(cls: ast.ClassDef):
        for n in cls.body:
            if isinstance(n, ast.FunctionDef) and n.name == "__init__":
                return n
        return None

    def get_init_body_text_local(init_func: ast.FunctionDef):
        full = get_source_segment_by_span_local(init_func)
        newline_idx = full.find("\n")
        if newline_idx == -1:
            return ""
        return full[newline_idx + 1 :]

    def unparse_expr_local(expr):
        try:
            return ast.unparse(expr)
        except Exception:
            if isinstance(expr, ast.Name):
                return expr.id
            elif isinstance(expr, ast.Attribute):
                return f"{unparse_expr_local(expr.value)}.{expr.attr}"
            else:
                return "<expr>"

    def get_base_class_string_local(cls: ast.ClassDef):
        bases = [unparse_expr_local(b) for b in cls.bases]
        return ", ".join(bases) if bases else ""

    for node in module_local.body:
        if isinstance(node, ast.ClassDef):
            class_name = node.name
            base_class = get_base_class_string_local(node)
            doc_block = get_class_docstring_block_local(node)
            init_func = find_init_func_local(node)
            if not init_func:
                continue
            full_block = get_source_segment_by_span_local(node)
            init_body = get_init_body_text_local(init_func)

            input_layer_name_match = re.search(r'self\.input_layer_name\s*=\s*"([^"]+)"', init_body)
            output_layer_name_match = re.search(r'self\.output_layer_name\s*=\s*"([^"]+)"', init_body)
            layer_type_match = re.search(r'self\.layer_type\s*=\s*([^\n]+)', init_body)
            output_codes_match = re.search(r'self\.output_codes_descriptions\s*=\s*(\{.*?\})', init_body, re.DOTALL)
            boundaries_match = re.search(r'self\.class_boundaries\s*=\s*(\{.*?\})', init_body, re.DOTALL)

            classes_local[class_name] = {
                "full_block": full_block,
                "match_span": span_to_slice_local(node.lineno, node.col_offset, node.end_lineno, node.end_col_offset),
                "class_name": class_name,
                "base_class": base_class,
                "docstring": doc_block,
                "init_body": init_body,
                "input_layer_name": input_layer_name_match.group(1) if input_layer_name_match else "",
                "output_layer_name": output_layer_name_match.group(1) if output_layer_name_match else "",
                "layer_type": layer_type_match.group(1).strip() if layer_type_match else "",
                "output_codes_descriptions": output_codes_match.group(1) if output_codes_match else "",
                "class_boundaries": boundaries_match.group(1) if boundaries_match else "",
            }

    if not classes_local:
        raise RuntimeError("No class with __init__ found while rebuilding classes.")
    return classes_local


# Extract classes once
classes = build_classes_from_text(source_text)
working_text = source_text
modification_log = []

In [ ]:
# Indentation helper
def detect_class_inner_indent(full_class_block: str) -> str:
    """
    Detect the indentation used inside the class by looking for its __init__ definition line.
    Returns whitespace (tabs or spaces) used before 'def __init__'.
    If not found, fall back to 4 spaces.
    """
    for line in full_class_block.splitlines():
        m = re.match(r"^([ \t]*)def\s+__init__\s*\(", line)
        if m:
            return m.group(1)
    return "    "

def indent_one_level(body: str, anchor_line: str) -> str:
    """
    Indent `body` one level deeper than the leading whitespace on `anchor_line`.
    Works for tabs or spaces; avoids hardcoded 4/8.
    """
    m = re.match(r"^([ \t]*)", anchor_line)
    base_ws = m.group(1) if m else ""

    if base_ws and set(base_ws) == {"\t"}:
        extra_ws = "\t"
    elif base_ws and set(base_ws) == {" "}:
        extra_ws = base_ws  # same width as existing level (one more level)
    else:
        extra_ws = " "  # fallback: single space

    normalized = textwrap.dedent(body.replace("\t", "    ")).strip() + "\n"
    return textwrap.indent(normalized, base_ws + extra_ws)

def black_format_block_with_scaffold(class_name: str, base: str, doc_raw: str, def_line: str, body_raw: str, class_inner_ws: str) -> str:
    """
    Build a minimal valid class block, run Black, and return the formatted block.
    The 'scaffold' is simply the class header + doc + def + body with consistent indents.
    """
    doc_norm = textwrap.dedent(doc_raw).strip() + "\n"
    doc_indented = textwrap.indent(doc_norm, class_inner_ws)
    body_indented = indent_one_level(body_raw, def_line)

    temp_src = f"class {class_name}({base}):\n" + doc_indented + def_line + body_indented
    mode = black.Mode(line_length=88)
    return black.format_str(temp_src, mode=mode)


In [ ]:
# UI widgets
class_selector = widgets.Dropdown(
    options=sorted(list(classes.keys())),
    description="Class :",
    layout=widgets.Layout(width="60%"),
)

base_options = [
    "l4_base.Level4ClassificationCatergoricalLayer",  # keep original spelling for compatibility
    "l4_base.Level4ClassificationContinuousLayer",
]
base_selector = widgets.Dropdown(
    options=base_options,
    description="Base type :",
    layout=widgets.Layout(width="60%"),
)

docstring_widget = widgets.Textarea(
    description="docstring:",
    layout=widgets.Layout(width="100%", height="150px"),
)

code_widgets = {}
desc_widgets = {}
out_rows = []
for k in LEVEL_KEYS:
    code_widgets[k] = widgets.Text(value="", placeholder=f"D{k}", layout=widgets.Layout(width="120px"))
    desc_widgets[k] = widgets.Text(value="", placeholder="description", layout=widgets.Layout(width="60%"))
    out_rows.append(widgets.HBox([widgets.Label(f"{k} :"), code_widgets[k], desc_widgets[k]]))
output_codes_box = widgets.VBox(out_rows)

min_widgets = {}
max_widgets = {}
bnd_rows = []
for k in LEVEL_KEYS:
    min_widgets[k] = widgets.Text(value="", placeholder="min or None", layout=widgets.Layout(width="140px"))
    max_widgets[k] = widgets.Text(value="", placeholder="max or None", layout=widgets.Layout(width="140px"))
    bnd_rows.append(widgets.HBox([widgets.Label(f"{k} :"), min_widgets[k], max_widgets[k]]))
boundaries_box = widgets.VBox(bnd_rows)

advanced_box = widgets.Accordion(children=[output_codes_box, boundaries_box])
advanced_box.set_title(0, "Output codes (per level)")
advanced_box.set_title(1, "Class boundaries (per level)")

apply_button = widgets.Button(description="Apply and write", button_style="success")
preview_button = widgets.Button(description="Preview", button_style="info")
reset_button = widgets.Button(description="Reset to default", button_style="warning")
output_area = widgets.Output()

In [ ]:
# Data collection + Conversion utilities
def collect_output_codes_from_ui():
    """
    Collect only filled rows into { code: (label, description) }.
    """
    data = {}
    for k in LEVEL_KEYS:
        label = code_widgets[k].value.strip()
        desc = desc_widgets[k].value.strip()
        if desc:
            if not label:
                label = f"D{k}"
            data[k] = (label, desc)
    return data

def parse_number_or_none(s: str):
    t = (s or "").strip()
    if not t:
        return None
    if t.lower() == "none":
        return None
    try:
        if "." in t or "e" in t.lower():
            return float(t)
        return int(t)
    except Exception:
        return None

def collect_boundaries_from_ui():
    """
    Collect only filled rows into { code: (min, max) }.
    """
    data = {}
    for k in LEVEL_KEYS:
        mn = parse_number_or_none(min_widgets[k].value)
        mx = parse_number_or_none(max_widgets[k].value)
        if mn is not None or mx is not None:
            data[k] = (mn, mx)
    return data

def set_boundaries_enabled(enabled: bool):
    for k in LEVEL_KEYS:
        min_widgets[k].disabled = not enabled
        max_widgets[k].disabled = not enabled

def build_modified_init_and_doc_raw(data, plan):
    """
    Return (init_body_raw, doc_block_raw) with *no* leading indentation.
    We normalize first, then apply edits, then insert boundaries WITHOUT preserving
    any previous indentation. Final indent happens later relative to def_line.
    """
    # Normalize raw init body BEFORE any edits to avoid mixed indent problems
    modified_init = textwrap.dedent(data["init_body"].replace("\t", "    ")).strip() + "\n"

    # Replace/insert input layer
    if re.search(r'self\.input_layer_name\s*=', modified_init):
        modified_init = re.sub(
            r'(self\.input_layer_name\s*=\s*)"[^"]*"',
            rf'\1"{plan["new_input"]}"',
            modified_init,
        )
    else:
        modified_init = f'self.input_layer_name = "{plan["new_input"]}"\n' + modified_init

    # Replace/insert output layer
    if re.search(r'self\.output_layer_name\s*=', modified_init):
        modified_init = re.sub(
            r'(self\.output_layer_name\s*=\s*)"[^"]*"',
            rf'\1"{plan["new_output"]}"',
            modified_init,
        )
    else:
        modified_init = f'self.output_layer_name = "{plan["new_output"]}"\n' + modified_init

    # Replace/insert layer type
    if re.search(r'self\.layer_type\s*=', modified_init):
        modified_init = re.sub(
            r'(self\.layer_type\s*=\s*)[^\n]+',
            rf'\1{plan["new_layer_type"]}',
            modified_init,
        )
    else:
        modified_init += f'\nself.layer_type = {plan["new_layer_type"]}\n'

    # Output codes
    out_dict = collect_output_codes_from_ui()
    if out_dict:
        pretty_out = format_output_codes_literal(out_dict, indent="    ")
        if re.search(r'self\.output_codes_descriptions\s*=\s*\{', modified_init, flags=re.DOTALL):
            modified_init = re.sub(
                r'(self\.output_codes_descriptions\s*=\s*)\{.*?\}',
                rf'\1{pretty_out}',
                modified_init,
                flags=re.DOTALL,
            )
        else:
            modified_init = f'self.output_codes_descriptions = {pretty_out}\n' + modified_init

    # Boundaries (raw insertion; no indent preservation)
    if plan["need_boundaries"]:
        bnd_dict = collect_boundaries_from_ui()
        pretty_bnd = format_boundaries_literal(bnd_dict, indent="    ") if bnd_dict else "{ }"
        modified_init = remove_existing_boundaries(modified_init)
        modified_init = insert_boundaries_after_output_codes(modified_init, pretty_bnd)
    else:
        modified_init = remove_existing_boundaries(modified_init)

    # Docstring (raw)
    doctext = docstring_widget.value.strip()
    if doctext:
        if not (doctext.startswith('"""') or doctext.startswith("'''")):
            doc_block = '"""\n' + doctext + '\n"""\n'
        else:
            doc_block = doctext.strip()
            if not doc_block.endswith("\n"):
                doc_block += "\n"
    else:
        doc_block = (data["docstring"] or "").strip()
        if doc_block and not doc_block.endswith("\n"):
            doc_block += "\n"

    return modified_init, doc_block

def compute_conversion_plan(class_name, target_base):
    data = classes[class_name]
    cur_name = data["class_name"]
    cur_input = data["input_layer_name"]
    cur_output = data["output_layer_name"]
    cur_layer_type = data["layer_type"]

    is_to_continuous = "ContinuousLayer" in target_base
    is_to_categorical = "CatergoricalLayer" in target_base or "CategoricalLayer" in target_base

    new_name = cur_name
    new_input = cur_input
    new_output = cur_output
    new_layer_type = cur_layer_type
    need_boundaries = False

    if is_to_continuous:
        new_name = cur_name.replace("L4a", "L4d")
        if cur_input:
            new_input = cur_input.replace("_cat", "_con")
        if cur_output:
            new_output = cur_output.replace("l4a", "l4d").replace("_cat", "_con")
        new_layer_type = "l4_base.ClassificationLayerTypes.DERIVATIVE"
        need_boundaries = True
    elif is_to_categorical:
        new_name = cur_name.replace("L4d", "L4a")
        if cur_input:
            new_input = cur_input.replace("_con", "_cat")
        if cur_output:
            new_output = cur_output.replace("l4d", "l4a").replace("_con", "_cat")
        new_layer_type = "l4_base.ClassificationLayerTypes.MAIN"
        need_boundaries = False

    return {
        "new_class_name": new_name,
        "new_input": new_input,
        "new_output": new_output,
        "new_layer_type": new_layer_type,
        "need_boundaries": need_boundaries,
    }

def remove_existing_boundaries(init_body):
    """
    Remove any existing class_boundaries block from the init body.
    """
    return re.sub(r"\n\s*self\.class_boundaries\s*=\s*\{.*?\}\s*", "\n", init_body, flags=re.DOTALL)

def insert_boundaries_after_output_codes(init_body, boundaries_text):
    """
    Insert boundaries right after the output codes assignment; otherwise append at end.
    IMPORTANT: Do NOT preserve legacy indentation here; we insert *raw* lines and
    let the final indent step (relative to def_line) make indentation consistent.
    """
    m = re.search(r"(self\.output_codes_descriptions\s*=\s*\{.*?\})", init_body, flags=re.DOTALL)
    if m:
        # Insert a raw boundaries line (no leading spaces)
        return init_body[: m.end()] + f"\nself.class_boundaries = {boundaries_text}\n" + init_body[m.end() :]
    else:
        return init_body + f"\nself.class_boundaries = {boundaries_text}\n"


In [ ]:
# UI callbacks (load and refresh)
def refresh_fields_from_selection(change=None):
    selected = class_selector.value
    if not selected:
        return
    data = classes[selected]

    # Docstring: show original block as-is
    docstring_widget.value = data["docstring"].strip() if data["docstring"] else ""

    # Reset codes and pre-fill
    for k in LEVEL_KEYS:
        code_widgets[k].value = ""
        desc_widgets[k].value = ""
    if data["output_codes_descriptions"]:
        try:
            out_dict = ast.literal_eval(data["output_codes_descriptions"])
            if isinstance(out_dict, dict):
                for k, v in out_dict.items():
                    if k in LEVEL_KEYS and isinstance(v, tuple) and len(v) == 2:
                        code_widgets[k].value = str(v[0])
                        desc_widgets[k].value = str(v[1])
        except Exception:
            pass

    # Reset boundaries and pre-fill
    for k in LEVEL_KEYS:
        min_widgets[k].value = ""
        max_widgets[k].value = ""
    if data["class_boundaries"]:
        try:
            bnd_dict = ast.literal_eval(data["class_boundaries"])
            if isinstance(bnd_dict, dict):
                for k, v in bnd_dict.items():
                    if k in LEVEL_KEYS and isinstance(v, tuple) and len(v) == 2:
                        mn, mx = v
                        min_widgets[k].value = "None" if mn is None else str(mn)
                        max_widgets[k].value = "None" if mx is None else str(mx)
        except Exception:
            pass

    # Sync base selector to current base
    if data["base_class"] in base_selector.options:
        base_selector.value = data["base_class"]
    else:
        opts = list(base_selector.options)
        if data["base_class"] not in opts:
            opts.append(data["base_class"])
        base_selector.options = opts
        base_selector.value = data["base_class"]

    plan = compute_conversion_plan(selected, base_selector.value)
    set_boundaries_enabled(plan["need_boundaries"])

def on_base_selector_change(change):
    selected = class_selector.value
    if not selected:
        return
    plan = compute_conversion_plan(selected, base_selector.value)
    set_boundaries_enabled(plan["need_boundaries"])

In [ ]:
# preview and apply
def on_preview_clicked(b):
    """
    Build the new class block, format with Black using a scaffold, show preview.
    """
    selected = class_selector.value
    if not selected:
        return
    data = classes[selected]
    plan = compute_conversion_plan(selected, base_selector.value)

    init_raw, doc_raw = build_modified_init_and_doc_raw(data, plan)

    # Detect class inner indent from the original class block when possible
    class_inner_ws = detect_class_inner_indent(data["full_block"])
    def_line = f"{class_inner_ws}def __init__(self):\n"

    # Black-format the class fragment via scaffold
    try:
        pretty = black_format_block_with_scaffold(
            class_name=plan["new_class_name"],
            base=base_selector.value,
            doc_raw=doc_raw,
            def_line=def_line,
            body_raw=init_raw,
            class_inner_ws=class_inner_ws,
        )
    except Exception as e:
        # Fall back to raw view (still indented relatively) if Black fails
        doc_indented = textwrap.indent(textwrap.dedent(doc_raw).strip() + "\n", class_inner_ws)
        body_indented = indent_one_level(init_raw, def_line)
        pretty = (
            f"Black formatting failed: {e}\n\n"
            f"class {plan['new_class_name']}({base_selector.value}):\n"
            + doc_indented
            + def_line
            + body_indented
        )

    with output_area:
        output_area.clear_output(wait=True)
        print("----- PREVIEW -----\n")
        print(pretty)
        print("\n-------------------")


def on_apply_clicked(b):
    """
    Build the new class block, splice it into working_text, format entire file,
    write to OUTPUT, then refresh classes and dropdown.
    """
    global working_text, classes

    selected = class_selector.value
    if not selected:
        return
    data = classes[selected]
    plan = compute_conversion_plan(selected, base_selector.value)

    if not modification_log:
        shutil.copyfile(SOURCE, SOURCE + ".bak")

    init_raw, doc_raw = build_modified_init_and_doc_raw(data, plan)
    class_inner_ws = detect_class_inner_indent(data["full_block"])
    def_line = f"{class_inner_ws}def __init__(self):\n"

    # Format the new class block first (clean injection), then splice
    clean_block = black_format_block_with_scaffold(
        class_name=plan["new_class_name"],
        base=base_selector.value,
        doc_raw=doc_raw,
        def_line=def_line,
        body_raw=init_raw,
        class_inner_ws=class_inner_ws,
    )

    start, end = data["match_span"]
    working_text = working_text[:start] + clean_block + working_text[end:]

    # Validate entire file, then format with Black
    try:
        ast.parse(working_text)
    except SyntaxError as e:
        with output_area:
            output_area.clear_output(wait=True)
            print(f"SyntaxError in generated file: {e}")
        return

    formatted_text = format_with_black(working_text)
    with open(OUTPUT, "w", encoding="utf-8") as f:
        f.write(formatted_text)

    # Rebuild metadata from the formatted text, update dropdown
    classes = build_classes_from_text(formatted_text)
    class_selector.options = sorted(list(classes.keys()))

    modification_log.append(f"Modifications for '{selected}' saved in {OUTPUT}")
    with output_area:
        output_area.clear_output(wait=True)
        for msg in modification_log:
            print(msg)

In [ ]:
# UI wire and display
def on_class_change(ch):
    """
    Rebuild classes from current working_text, update dropdown, refresh widgets.
    """
    global classes
    classes = build_classes_from_text(working_text)
    class_selector.options = sorted(list(classes.keys()))
    refresh_fields_from_selection(ch)

class_selector.observe(on_class_change, names="value")
base_selector.observe(on_base_selector_change, names="value")
apply_button.on_click(on_apply_clicked)
preview_button.on_click(on_preview_clicked)

def on_reset_clicked(b):
    """
    Reset to ORIGINAL source: reload, rebuild classes, update dropdown, refresh UI.
    """
    global working_text, classes
    with open(SOURCE, "r", encoding="utf-8") as f:
        source_text_fresh = f.read()
    working_text = source_text_fresh
    classes = build_classes_from_text(source_text_fresh)
    class_selector.options = sorted(list(classes.keys()))
    refresh_fields_from_selection()

reset_button.on_click(on_reset_clicked)

# Initial fill
if class_selector.value:
    refresh_fields_from_selection()

controls = widgets.VBox(
    [
        widgets.HBox([class_selector, base_selector]),
        docstring_widget,
        advanced_box,  # per-level editors
        widgets.HBox([preview_button, apply_button, reset_button]),
        output_area,
    ]
)

display(controls)